In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_object as go
colors = ['#082040', '#175073', '3285A6', '#B8D0D9', '#6CC5D9']

In [53]:
import pickle

In [54]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_2.csv')

In [55]:
print(df.shape, df.info(), sep=', ', end='\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.1        533 non-null    int64  
 1   volume              533 non-null    float64
 2   area                533 non-null    float64
 3   Unnamed: 0          533 non-null    int64  
 4   c1                  533 non-null    object 
 5   c2                  322 non-null    object 
 6   c3                  9 non-null      object 
 7   chemical_formula    533 non-null    object 
 8   crystal_structure   533 non-null    object 
 9   z                   509 non-null    float64
 10  id                  533 non-null    int64  
 11  space_group_shell   533 non-null    int64  
 12  structure           533 non-null    object 
 13  structure_          533 non-null    object 
 14  formula_combined    533 non-null    object 
 15  coer_oe             533 non-null    float64
 16  exc_bias

In [56]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'c1', 'c2', 'c3', 'structure_']) # Колонку structure - нужно будет как-то закодировать 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   volume              533 non-null    float64
 1   area                533 non-null    float64
 2   chemical_formula    533 non-null    object 
 3   crystal_structure   533 non-null    object 
 4   z                   509 non-null    float64
 5   id                  533 non-null    int64  
 6   space_group_shell   533 non-null    int64  
 7   structure           533 non-null    object 
 8   formula_combined    533 non-null    object 
 9   coer_oe             533 non-null    float64
 10  exc_bias_oe         533 non-null    float64
 11  fc_field_t          533 non-null    float64
 12  h_range_max_koe     533 non-null    float64
 13  y                   533 non-null    float64
 14  mr (emu/g)          533 non-null    float64
 15  sat_em_g            533 non-null    float64
 16  shape   

Столбец structure имеет в себе формулы с структурой формулы ядро/оболочка, поэтому она для модели не будет иметьсмысла, но для интерпритации важна, поэтому совместим её знчаения с колонкой id, которая тоже не несёт никакого знания о наносатицах напрямую. Создадим словарь, где каждый id - structure

In [57]:
print(len(df['id'].unique())) # Эта колнка состоит только из уникальных значений

533


In [58]:
dict_structure = {} # Словарь для кодирования
# Проходимся по каждой строке DataFrame и добавляем в словарь пару id-structure
for idx, row in df.iterrows():
    dict_structure[row['id']] = row['chemical_formula']

# Пример значения id, для которого мы хотим декодировать structure
id_value = 123

In [59]:
# Сохраняем словарь в файл с помощью pickle(пакет помогает сохранять данные и загружать обратно)
with open('dict_structure.pkl', 'wb') as f:
    pickle.dump(dict_structure, f)

In [ ]:
# Проверка
decoded_structure = dict_structure[id_value]
print(decoded_structure)

['FeO', 'Fe3O4']


In [ ]:
del df['structure']
del df['chemical_formula']

In [ ]:
# Преобразуем crystal_structure в цифры


In [ ]:
print(df['crystal_structure'].unique()) # Разделиим и стандартизируемм значения, а потом перведём их в цифры и снова склеим

['hexagonal, cubic ' 'cubcic, cube' 'cubic' 'orthorhombic' 'cubic, cubic'
 'trigonal' 'cubic, trigonal' 'tetragonal' 'cubic, monoclinic'
 'cube, cube' 'trigonal, orthorhombic' 'tetragonal, cubic' 'cube'
 'cubic, orthorhombic' 'cubic,  cubic' 'triclinic,cubic ' 'hexagonal'
 'hexagonal, cubic' 'cubic, tetrahedral' 'cubic ' 'cubic, tetragonal'
 'orthorhombic, cubic' 'cubic,  trigonal' 'monoclinic, cubic' 'monoclinic'
 'cubic, hexagonal']


In [ ]:
dict_crystal_str = {
    'triclinic': 1, 'monoclinic': 2, 'orthorhombic': 3,
    'tetragonal': 4, 'trigonal': 5, 'hexagonal': 6,
    'cubic': 7, 'cube': 7
}

In [ ]:
# Функция для кодирования списка значений в соответствии с словарем
def encode_crystal_list(crystal_list):
    return [dict_crystal_str[crystal] for crystal in crystal_list]

In [ ]:
# Применяем функцию к столбцу с списками
df['encoded_crystal_list'] = df['crystal_structure'].apply(lambda x: [dict_crystal_str[item] for item in x])
print(df)

KeyError: 'h'

In [ ]:
df.isna()
df = df.ddrop_dublicates(subset=['id'], keep=False, inplace=True)

,volume,area,crystal_structure,z,id,space_group_shell,formula_combined,coer_oe,exc_bias_oe,fc_field_t,...,shape,space_group_core,temperature_k,ver_shift_emu_g,x,mass_ratio,magn_ions_combined,num_of_unique_magn_sites,sphericity,total_magn
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
529,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
530,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
531,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False



KeyboardInterrupt



In [ ]:
print(df.info())

In [ ]:
subset_df = df.loc[:]
correlation_matric - subset_df.corr()

# Построение матрицы корреляции
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('')
plt.show()